In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib widget

In [ ]:
def read_report(path):
    df = pd.read_csv(path)
    df['ts'] = df.apply(lambda x: pd.Timestamp(x['תאריך קריאה'] + " " + x['שעת קריאה']), axis=1)
    df.rename(columns={'צריכה': 'consumption'}, inplace=True)
    df.set_index('ts', inplace=True, drop=True)
    dropped_columns = df.columns.tolist(); dropped_columns.remove('consumption')
    df.drop(columns=dropped_columns, axis=1, inplace=True)
    return df

In [ ]:
df = read_report('power.csv')
df['day_name'] = df.index.day_name()
df.consumption.plot()

In [ ]:
default_rate = 0.6
pazgaz_plans = [
    {'name': 'unlimited', 'discount': 0.05, 'periods':  
                                                    [
                                                    {"day": day, "start":pd.Timestamp('00:00:00'), "end": pd.Timestamp('23:59:59')} for day in ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
                                                    ]
    },
    {'name': 'weekend', 'discount': 0.1, 'periods':
                                                [
                                                {"day": day, "start":pd.Timestamp('00:00:00'), "end": pd.Timestamp('23:59:59')} for day in ["Friday", "Saturday"]
                                                ]
    },
    {'name': 'day', 'discount': 0.15, 'periods':  
                                                [
                                                {"day": day, "start":pd.Timestamp('08:00:00'), "end": pd.Timestamp('15:59:59')} for day in ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
                                                ]
    },
    {'name': 'night', 'discount': 0.15, 'periods':
                                                [
                                                {"day": day, "start":pd.Timestamp('00:00:00'), "end": pd.Timestamp('06:59:59')} for day in ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
                                                ] 
                                                + 
                                                [
                                                {"day": day, "start":pd.Timestamp('23:00:00'), "end": pd.Timestamp('23:59:59')} for day in ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
                                                ]
    }
]

In [ ]:
total_consumption = df.consumption.sum()
print("Total consumption: %d KW"%(total_consumption))
print('Cost: %.2f NIS' %(total_consumption * default_rate))

In [ ]:
def apply_plan_discount(df, plan):
    total_consumption = df.consumption.sum()
    discunted_consumption = 0
    for period in plan['periods']:
        discunted_consumption += df[df.day_name == period['day']].between_time(period['start'].time(), period['end'].time()).consumption.sum()
    print(f"Plan: {plan['name']}")
    print("Discunted consumption: %d KW" %(discunted_consumption))
    print("Cost: %.2f NIS" % ((total_consumption - discunted_consumption * plan['discount']) * default_rate))
    print("")

In [ ]:
for plan in pazgaz_plans:
    apply_plan_discount(df, plan)